In [ ]:
import pandas as pd

#Loading the dataset and lowercase all column names
file_path = "combine_VPD_data_2024-04-24.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")
df.columns = df.columns.str.lower()  # normalize column names

df.head(10)


In [ ]:
# checking missing values in the columns
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]
print(missing_values)


In [ ]:
# Since it is only province which has missing values, we are excluding these rows where 'province_name' is missing
df_cleaned = df[df['province_name'].notna()]

# check that missing values in 'province_name' are gone
print("Missing value is:", df_cleaned['province_name'].isnull().sum())  # Should be 0

df_cleaned


In [ ]:
#filtering the columns to keep
columns_to_keep = [
    'year', 'month','cname', 'country_code', 'province_name', 'district_name', 'region',
    'bcg_l1', 'rota1_l1', 'rota2_l1', 'measles_l1',
    'opv1_l1', 'opv3_l1', 'livebirths', 'survivinginfants','numhfreportsincluded', 'districtpopulation'
]

df_vaccines = df_cleaned[columns_to_keep]

df_vaccines


In [ ]:
# making a clean copy to avoid SettingWithCopyWarning
df_vaccines = df_vaccines.copy()

df_vaccines.loc[:, 'bcg_coverage'] = (df_vaccines['bcg_l1'] / df_vaccines['survivinginfants']) * 100
df_vaccines.loc[:, 'measles_coverage'] = (df_vaccines['measles_l1'] / df_vaccines['survivinginfants']) * 100
df_vaccines.loc[:, 'opv1_coverage'] = (df_vaccines['opv1_l1'] / df_vaccines['survivinginfants']) * 100
df_vaccines.loc[:, 'opv3_coverage'] = (df_vaccines['opv3_l1'] / df_vaccines['survivinginfants']) * 100
df_vaccines.loc[:, 'rota1_coverage'] = (df_vaccines['rota1_l1'] / df_vaccines['survivinginfants']) * 100
df_vaccines.loc[:, 'rota2_coverage'] = (df_vaccines['rota2_l1'] / df_vaccines['survivinginfants']) * 100


df_vaccines


In [ ]:
Checking the Column List
print(df_vaccines.columns.tolist())


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define coverage columns
coverage_columns = [
    'bcg_coverage', 'measles_coverage', 'opv1_coverage',
    'opv3_coverage', 'rota1_coverage', 'rota2_coverage'
]

# Melt the data to long format
df_long = df_vaccines.melt(
    id_vars=['year', 'cname', 'country_code', 'survivinginfants'],
    value_vars=coverage_columns,
    var_name='antigen',
    value_name='coverage'
)

#Clean and convert coverage back to absolute vaccinated count
df_long = df_long.dropna(subset=['coverage', 'survivinginfants'])
df_long['year'] = pd.to_numeric(df_long['year'], errors='coerce')
df_long['vaccinated'] = (df_long['coverage'] / 100) * df_long['survivinginfants']

#Group and calculate true weighted average coverage per country/year/antigen
grouped = df_long.groupby(['year', 'cname', 'country_code', 'antigen']).agg({
    'vaccinated': 'sum',
    'survivinginfants': 'sum'
}).reset_index()

grouped['weighted_avg_coverage'] = (grouped['vaccinated'] / grouped['survivinginfants']) * 100
grouped['weighted_avg_coverage'] = grouped['weighted_avg_coverage'].round(1)

Get top/bottom countries based on mean across all years
avg_country_coverage = (
    grouped.groupby('cname')['weighted_avg_coverage']
    .mean()
    .sort_values()
)

top_countries = avg_country_coverage.tail(6).index.tolist()
bottom_countries = avg_country_coverage.head(6).index.tolist()

print("Top 6 Countries by Avg Weighted Coverage:", top_countries)
print("Bottom 6 Countries by Avg Weighted Coverage:", bottom_countries)

#Plot Top Countries
df_top = grouped[grouped['cname'].isin(top_countries)]

if not df_top.empty:
    g_top = sns.relplot(
        data=df_top,
        x="year", y="weighted_avg_coverage", kind="line",
        hue="cname", col="antigen", col_wrap=3,
        height=4, aspect=1.5, facet_kws={"sharey": False}
    )
    g_top.fig.subplots_adjust(top=0.9)
    g_top.fig.suptitle("Top 6 Countries by Weighted Avg Coverage", fontsize=16)
    plt.show()
else:
    print("No data found for top countries.")

#Plot Bottom Countries
df_bottom = grouped[grouped['cname'].isin(bottom_countries)]

if not df_bottom.empty:
    g_bottom = sns.relplot(
        data=df_bottom,
        x="year", y="weighted_avg_coverage", kind="line",
        hue="cname", col="antigen", col_wrap=3,
        height=4, aspect=1.5, facet_kws={"sharey": False}
    )
    g_bottom.fig.subplots_adjust(top=0.9)
    g_bottom.fig.suptitle("Bottom 6 Countries by Weighted Avg Coverage", fontsize=16)
    plt.show()
else:
    print("No data found for bottom countries.")


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Latest measles data
latest_measles = (
    df_vaccines.sort_values('year')
    .groupby('country_code')
    .tail(1)
    [['country_code', 'cname', 'measles_coverage']]
)

# Create the base choropleth
fig = px.choropleth(
    latest_measles,
    locations="country_code",
    color="measles_coverage",
    hover_name="cname",
    color_continuous_scale="YlOrRd",
    title="Measles Vaccine Coverage in Africa (with Country Labels)",
    range_color=(0, 100),
    projection="natural earth"
)

# Zoom in on Africa
fig.update_geos(
    visible=False,
    scope="africa",
    showcountries=True,
    showframe=False
)

# Add static text labels for countries (manually from centroids)
# Plotly doesn't provide these directly, so we extract from the data
# Get ISO3 code + centroids for labeling
import pycountry
import geopy
from geopy.geocoders import Nominatim
import time

# Use geopy to get approximate country centroids
geolocator = Nominatim(user_agent="geoapi")

annotations = []
for row in latest_measles.itertuples():
    try:
        location = geolocator.geocode(row.cname)
        if location:
            annotations.append(
                go.layout.Annotation(
                    text=row.cname,
                    x=location.longitude,
                    y=location.latitude,
                    xref='x', yref='y',
                    showarrow=False,
                    font=dict(size=9, color="black")
                )
            )
        time.sleep(1)  # To avoid request limit
    except:
        continue

# Final layout
fig.update_layout(
    width=1100,
    height=700,
    margin={"r":0,"t":50,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Coverage (%)",
        ticksuffix="%",
        lenmode="fraction", len=0.75
    ),
    annotations=annotations
)

fig.show()


In [ ]:
#Compute national average coverage by country and year
national_avg = (
    df_vaccines.groupby(['year', 'cname'])['measles_coverage']
    .mean()
    .reset_index()
    .rename(columns={'measles_coverage': 'national_avg'})
)

#Merge national average back into district-level data
df_measles = df_vaccines.merge(national_avg, on=['year', 'cname'], how='left')

#Calculate coverage gap and flag underperforming districts
df_measles['gap'] = df_measles['national_avg'] - df_measles['measles_coverage']
df_measles['flag'] = df_measles['gap'] > 10  # Customize this threshold if needed

#Filter and show only the underperforming districts
underperforming = df_measles[df_measles['flag'] == True][
    ['year', 'cname', 'province_name', 'district_name',
     'measles_coverage', 'national_avg', 'gap']
].sort_values(by='gap', ascending=False)

#Display top 10 underperforming districts
print("Top Underperforming Districts for Measles Coverage:\n")
print(underperforming.head(10).to_string(index=False))


In [ ]:

# Dropout Rate Calculations

# OPV Dropout: OPV1 , OPV3
df_vaccines['opv_dropout'] = df_vaccines.apply(
    lambda row: ((row['opv1_coverage'] - row['opv3_coverage']) / row['opv1_coverage']) * 100
    if pd.notnull(row['opv1_coverage']) and row['opv1_coverage'] > 0 else None,
    axis=1
)

# Rota Dropout: Rota1 , Rota2
df_vaccines['rota_dropout'] = df_vaccines.apply(
    lambda row: ((row['rota1_coverage'] - row['rota2_coverage']) / row['rota1_coverage']) * 100
    if pd.notnull(row['rota1_coverage']) and row['rota1_coverage'] > 0 else None,
    axis=1
)

# BCG to Measles Dropout: BCG , Measles1
df_vaccines['bcg_measles_dropout'] = df_vaccines.apply(
    lambda row: ((row['bcg_coverage'] - row['measles_coverage']) / row['bcg_coverage']) * 100
    if pd.notnull(row['bcg_coverage']) and row['bcg_coverage'] > 0 else None,
    axis=1
)

# Round results for clarity
df_vaccines['opv_dropout'] = df_vaccines['opv_dropout'].round(1)
df_vaccines['rota_dropout'] = df_vaccines['rota_dropout'].round(1)
df_vaccines['bcg_measles_dropout'] = df_vaccines['bcg_measles_dropout'].round(1)

# Preview
print(df_vaccines[['year', 'cname', 'district_name', 
                   'opv_dropout', 'rota_dropout', 'bcg_measles_dropout']].head())


In [ ]:
# Feature Engineering and Selection: 

# Develop composite indices for healthcare accessibility and socio-economic status. 
# Use feature importance techniques (Random Forest feature importance) to identify key predictors of low coverage. 

from sklearn.preprocessing import MinMaxScaler

# Selecting the proxy columns
healthcare_proxies = ['numhfreportsincluded', 'survivinginfants', 'districtpopulation']

# Removing rows with missing data
df_features = df_vaccines.dropna(subset=healthcare_proxies).copy()

# Normalize (0–1 scale)
scaler = MinMaxScaler()
df_features[healthcare_proxies] = scaler.fit_transform(df_features[healthcare_proxies])

# Composite Index (equal weighting)
df_features['healthcare_access_index'] = df_features[healthcare_proxies].mean(axis=1)


In [ ]:
df_features['low_measles_coverage'] = df_vaccines['measles_coverage'] < 80  # Binary classification


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

features = ['healthcare_access_index', 'survivinginfants', 'districtpopulation']
target = 'low_measles_coverage'

X = df_features[features]
y = df_features[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Feature Importance
importances = pd.Series(clf.feature_importances_, index=features).sort_values(ascending=False)

print("Feature Importance:")
print(importances)


Prepares data: Normalizes healthcare access features.

Creates a binary target for each antigen → low_ANTIGEN (True if coverage < 80).

Trains a Random Forest classifier for each antigen.

Calculates feature importances (global, average impact across all predictions).

Plots horizontal bar charts to visualize which features are most predictive of low coverage.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# defining coverage columns
coverage_columns = [
    'bcg_coverage', 'rota1_coverage', 'rota2_coverage',
    'measles_coverage', 'opv1_coverage', 'opv3_coverage'
]
proxy_columns = ['numhfreportsincluded', 'survivinginfants', 'districtpopulation']

# Preparing Dataset
df_features = df_vaccines.dropna(subset=proxy_columns + coverage_columns).copy()

# Normalizing proxy features
scaler = MinMaxScaler()
df_features[proxy_columns] = scaler.fit_transform(df_features[proxy_columns])
df_features['healthcare_access_index'] = df_features[proxy_columns].mean(axis=1)

# Final list of features to use in model
model_features = ['healthcare_access_index', 'survivinginfants', 'districtpopulation']

# Run Random Forest for Each Antigen and Plot
for antigen in coverage_columns:
    target = f"low_{antigen}"
    df_features[target] = df_features[antigen] < 80  # Binary target

    X = df_features[model_features]
    y = df_features[target]

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    importances = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=True)

    # Plot feature importance
    plt.figure(figsize=(8, 4))
    importances.plot(kind='barh')
    plt.title(f"Feature Importance for Low {antigen.replace('_coverage', '').upper()} Coverage")
    plt.xlabel("Importance Score")
    plt.tight_layout()
    plt.grid(axis='x', linestyle='--', alpha=0.5)
    plt.show()


Reuses the last fitted model (RandomForestClassifier).

Uses SHAP (SHapley Additive exPlanations) to compute local feature contributions.

Generates a summary plot to show how each feature impacts predictions across all samples.

Each dot = a district

X-axis = impact on prediction

Color = feature value (red = high, blue = low)

In [ ]:
import shap

# Fit model as usual
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Create SHAP explainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# Summary plot (for class 1: low coverage)
shap.summary_plot(shap_values[1], X_train)


In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1][0], X_train.iloc[0])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, auc
from imblearn.over_sampling import SMOTE

# Configurating proxy columns and model features
proxy_columns = ['numhfreportsincluded', 'survivinginfants', 'districtpopulation']
model_features = proxy_columns + ['healthcare_access_index']

# Normalizing healthcare access features once
scaler = MinMaxScaler()
df_vaccines[proxy_columns] = scaler.fit_transform(df_vaccines[proxy_columns])
df_vaccines['healthcare_access_index'] = df_vaccines[proxy_columns].mean(axis=1)

# Define reusable function
def run_classification_model(antigen_column):
    print(f"\n Running classification for: {antigen_column}")

    # Create binary target
    target_col = f"high_{antigen_column}"
    df_vaccines[target_col] = df_vaccines[antigen_column] >= 90

    # Filter and drop NA
    df_model = df_vaccines.dropna(subset=model_features + [target_col]).copy()
    X = df_model[model_features]
    y = df_model[target_col]

    # Show class distribution
    print("Class distribution:\n", y.value_counts())

    # Apply SMOTE
    sm = SMOTE(random_state=42)
    X_resampled, y_resampled = sm.fit_resample(X, y)

    # Train/Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_resampled, y_resampled, stratify=y_resampled, test_size=0.2, random_state=42
    )

    # Train Classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Evaluate
    y_pred = clf.predict(X_test)
    print("\n Classification Report:")
    print(classification_report(y_test, y_pred))

    # Plot ROC Curve
    y_probs = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve: High {antigen_column.replace('_coverage', '').upper()} Coverage")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
antigen_list = [
    'bcg_coverage', 'rota1_coverage', 'rota2_coverage',
    'measles_coverage', 'opv1_coverage', 'opv3_coverage'
]

for antigen in antigen_list:
    run_classification_model(antigen)


In [ ]:
print("Real test set distribution:")
print(y_test.value_counts())


In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from imblearn.over_sampling import SMOTE

# reusable classification function with ROC + SMOTE
def run_classification_all_antigens(df, antigen_columns, feature_columns, threshold=90):
    results = {}
    auc_scores = {}

    # Normalize and add healthcare access index
    scaler = MinMaxScaler()
    df[feature_columns] = scaler.fit_transform(df[feature_columns])
    df['healthcare_access_index'] = df[feature_columns].mean(axis=1)
    features = feature_columns + ['healthcare_access_index']

    for antigen in antigen_columns:
        target_col = f"high_{antigen}"
        print(f"\nRunning classification for: {antigen}")

        # Create binary target high coverage or not
        df[target_col] = df[antigen] >= threshold

        # Drop missing values
        df_model = df.dropna(subset=features + [target_col]).copy()
        X = df_model[features]
        y = df_model[target_col]

        print("Class distribution:\n", y.value_counts())

        # Split first (retain imbalance in test set)
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

        # SMOTE only on training set
        smote = SMOTE(random_state=42)
        X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

        # Train model
        clf = RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
        clf.fit(X_train_sm, y_train_sm)

        # Predict and evaluate
        y_pred = clf.predict(X_test)
        print("\n Classification Report (imbalanced test set):")
        print(classification_report(y_test, y_pred))

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        ConfusionMatrixDisplay(cm).plot()
        plt.title(f"Confusion Matrix: {target_col}")
        plt.grid(False)
        plt.tight_layout()
        plt.show()

        # ROC Curve
        y_probs = clf.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_probs)
        roc_auc = auc(fpr, tpr)

        plt.figure(figsize=(6, 5))
        plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve: {antigen.replace('_coverage', '').upper()}")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

        # Save results
        results[antigen] = {
            'model': clf,
            'report': classification_report(y_test, y_pred, output_dict=True),
            'confusion_matrix': cm,
            'roc_auc': roc_auc
        }

        auc_scores[antigen] = roc_auc

    return results, auc_scores

In [ ]:
# Running classification for all 6 antigens
coverage_columns = [
    'bcg_coverage', 'rota1_coverage', 'rota2_coverage',
    'measles_coverage', 'opv1_coverage', 'opv3_coverage'
]

proxy_columns = ['numhfreportsincluded', 'survivinginfants', 'districtpopulation']

results, auc_scores = run_classification_all_antigens(df_vaccines, coverage_columns, proxy_columns)

**Insights into Immunization Coverage**

-A table of regions (districts) with lowest average coverage.

-A table of highest dropout rates between doses (e.g. OPV1 , OPV3).

-Bar plots to visualize primary influencing factors (from Random Forest importances).

**Low Coverage Table & Plot**

In [ ]:
# List of all antigens to check
coverage_columns = [
    'bcg_coverage', 'measles_coverage', 'opv1_coverage',
    'opv3_coverage', 'rota1_coverage', 'rota2_coverage'
]

# Number of lowest performing districts to display per antigen
N = 15

low_coverage_tables = {}

for antigen in coverage_columns:
    # Group by and compute mean coverage
    table = (
        df_vaccines.groupby(['cname', 'province_name', 'district_name'])[antigen]
        .mean()
        .sort_values(ascending=False
        .head(N)
        .reset_index()
    )
    low_coverage_tables[antigen] = table

# Display each table
for antigen, table in low_coverage_tables.items():
    print(f"\n Bottom {N} Districts by {antigen.replace('_coverage', '').upper()} Coverage:")
    display(table.style.background_gradient(cmap='OrRd'))



**Dropout Rate (e.g., OPV1 → OPV3)**

In [ ]:
# Define antigen series pairs
dropout_pairs = {
    'opv_dropout_rate': ('opv1_l1', 'opv3_l1'),
    'rota_dropout_rate': ('rota1_l1', 'rota2_l1'),
    'measles_dropout_rate': ('bcg_l1', 'measles_l1') 
}

dropout_tables = {}

for dropout_name, (dose1, dose2) in dropout_pairs.items():
    # Calculate dropout rate
    df_vaccines[dropout_name] = ((df_vaccines[dose1] - df_vaccines[dose2]) / df_vaccines[dose1]) * 100

    # Filter out divisions where dose1 is 0 to avoid division by zero
    filtered_df = df_vaccines[df_vaccines[dose1] > 0]

    # Group and summarize
    table = (
        filtered_df.groupby(['cname', 'province_name', 'district_name'])[dropout_name]
        .mean()
        .sort_values(ascending=False)
        .head(10)
        .reset_index()
    )

    # Save each table
    dropout_tables[dropout_name] = table

# Display each dropout table with styled gradient
for name, table in dropout_tables.items():
    print(f"\n Top 10 Districts with Highest {name.replace('_', ' ').upper()}:")
    display(table.style.background_gradient(cmap='YlOrRd'))



**Predictive Models**

We'll visualize model performance for forecasting “high coverage” using ROC curves and AUCs.

**Summary Table of AUCs**

In [ ]:
# display AUC summary table
auc_df = pd.DataFrame(auc_scores.items(), columns=['Antigen', 'AUC'])
auc_df.sort_values(by='AUC', ascending=False).style.bar(subset=['AUC'], color='lightgreen')


**Geospatial Visualizations**

We’ll use Plotly to make choropleth maps

In [ ]:
import plotly.graph_objects as go

# Coverage columns to visualize
antigen_coverage_columns = [
    'bcg_coverage', 'rota1_coverage', 'rota2_coverage',
    'measles_coverage', 'opv1_coverage', 'opv3_coverage'
]

# Filter only rows with required data
df_plot = df_vaccines[['year', 'country_code', 'cname'] + antigen_coverage_columns].dropna()

# Get unique years and sort
years = sorted(df_plot['year'].unique())

# Creating traces (antigen × year)
data = []
buttons_antigen = []
buttons_year = []
traces_visibility = []

# Loop through antigens and years to build traces
for antigen in antigen_coverage_columns:
    for year in years:
        subset = df_plot[df_plot['year'] == year]
        trace = go.Choropleth(
            locations=subset['country_code'],
            z=subset[antigen],
            text=subset['cname'],
            colorscale='RdYlGn',
            zmin=0,
            zmax=100,
            colorbar_title='Coverage (%)',
            geo='geo',
            visible=False,
            name=f"{antigen}_{year}"
        )
        data.append(trace)
        traces_visibility.append((antigen, year))

# Initial visibility (first antigen and year)
initial_antigen, initial_year = traces_visibility[0]
for i, (a, y) in enumerate(traces_visibility):
    data[i].visible = (a == initial_antigen and y == initial_year)

# Creating antigen dropdown buttons
for antigen in antigen_coverage_columns:
    visibility = [(a == antigen and y == initial_year) for a, y in traces_visibility]
    buttons_antigen.append(dict(
        label=antigen.replace('_coverage', '').upper(),
        method='update',
        args=[{'visible': visibility},
              {'title': f"{antigen.replace('_coverage', '').upper()} Coverage in Africa - Year {initial_year}"}]
    ))

# Creating year dropdown buttons
for year in years:
    visibility = [(a == initial_antigen and y == year) for a, y in traces_visibility]
    buttons_year.append(dict(
        label=str(year),
        method='update',
        args=[{'visible': visibility},
              {'title': f"{initial_antigen.replace('_coverage', '').upper()} Coverage in Africa - Year {year}"}]
    ))

# Layout
layout = go.Layout(
    title=f"{initial_antigen.replace('_coverage', '').upper()} Coverage in Africa - Year {initial_year}",
    geo=dict(scope='africa'),
    width=1000,
    height=700,
    margin=dict(l=40, r=40, t=80, b=40),
    updatemenus=[
        dict(
            buttons=buttons_antigen,
            direction='down',
            showactive=True,
            x=0.25, y=1.12, xanchor='center', yanchor='top'
        ),
        dict(
            buttons=buttons_year,
            direction='down',
            showactive=True,
            x=0.75, y=1.12, xanchor='center', yanchor='top'
        )
    ]
)

# Final figure
fig = go.Figure(data=data, layout=layout)
fig.show()


In [ ]:
#Preparing the latest year per country for each antigen
def prepare_facet_data(df, antigen):
    df_antigen = df[['year', 'country_code', 'cname', antigen]].dropna()
    df_antigen = df_antigen.sort_values('year')
    
    # Group by country & year to get one row per (country, year)
    df_antigen_grouped = df_antigen.groupby(['year', 'country_code', 'cname'])[antigen].mean().reset_index()
    return df_antigen_grouped

# Looping through all antigens and create maps
antigens = [
    'bcg_coverage', 'rota1_coverage', 'rota2_coverage',
    'measles_coverage', 'opv1_coverage', 'opv3_coverage'
]

for antigen in antigens:
    df_facet = prepare_facet_data(df_vaccines, antigen)

    fig = px.choropleth(
        df_facet,
        locations='country_code',
        color=antigen,
        hover_name='cname',
        animation_frame='year',
        color_continuous_scale='RdYlGn',
        range_color=(0, 100),
        scope='africa',
        title=f"{antigen.replace('_coverage', '').upper()} Coverage by Year in Africa"
    )

    fig.update_layout(
        width=1000,
        height=700,
        margin=dict(r=0, l=0, t=50, b=0),
        coloraxis_colorbar=dict(title="Coverage (%)")
    )

    fig.show()
